#### DATA :

+ 2000 recordings/ 10 class
+ Save at path = "recordings"

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
import wave
from sklearn.model_selection import train_test_split
import librosa as lib

In [13]:
data = []
labels = []
path = 'recordings'
coef = 0
count = 0
for file_name in os.listdir(path):
    count += 1
    w,f = lib.core.load(os.path.join(path,file_name))
    feature = np.mean(lib.feature.mfcc(y = w, sr= f,n_mfcc= 40).T, axis = 0)
    data.append(feature)
    labels.append(coef)
    if(count % 200 == 0):
        coef += 1

In [4]:
data = np.asanyarray(data)
labels = np.asanyarray(labels)
print("Number of voice datas: ",data.shape )
print("Shape of labels : ", labels.shape)


X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.25)

print("Number of data train : ", X_train.shape)
print("Labels of data train : ", y_train.shape)
print("Number of data test : ", X_test.shape)
print("Labels of data test : ", y_test.shape)

Number of voice datas:  (2000, 40)
Shape of labels :  (2000,)
Number of data train :  (1500, 40)
Labels of data train :  (1500,)
Number of data test :  (500, 40)
Labels of data test :  (500,)


In [13]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(250,250,300),max_iter= 20000).fit(X_train, y_train)

In [14]:
y_pre = mlp.predict(X_test)
from sklearn.metrics import accuracy_score
print("Hieu qua mo hinh dat :", 100* accuracy_score(y_test, y_pre.tolist() ) )

Hieu qua mo hinh dat : 97.2


In [15]:
from sklearn.neural_network import MLPClassifier
mlp1 = MLPClassifier(hidden_layer_sizes=(300,300,300),max_iter= 20000).fit(X_train, y_train)
y_pre = mlp1.predict(X_test)
from sklearn.metrics import accuracy_score
print("Hieu qua mo hinh dat :", 100* accuracy_score(y_test, y_pre.tolist() ) )

Hieu qua mo hinh dat : 95.19999999999999


Phát hiện các số được nói khi nói một dãy số:

In [16]:
# SAVE MODEL 
import pickle
file_name = "model_MLP.sav"
pickle.dump(mlp, open(file_name, 'wb'))

In [2]:
#load model
import pickle
file_name = "model_MLP.sav"
mlp = pickle.load(open(file_name, 'rb'))

In [17]:
y_pre = mlp.predict(X_test)
from sklearn.metrics import accuracy_score
print("Hieu qua mo hinh dat :", 100* accuracy_score(y_test, y_pre.tolist() ) )

Hieu qua mo hinh dat : 97.2


In [3]:
path2 = 'file_mp3'
for file_name in os.listdir(path2):
    sound, f_sound = lib.core.load(os.path.join(path2, file_name))

In [47]:
stride = int(0.07 * f_sound)
pos = 0
engry = []
before = True
after = False
shot_voice = []
while pos < sound.shape[0]:
    
    temp = sound[pos : pos+ stride]
    engry.append(np.mean(np.sum(temp ** 2)))
    pos = pos + stride 
    if pos > sound.shape[0]:
        break
    #temp_after = sound[pos : pos+ stride] 
print("Engry of 0.15: ", engry)    
print("Mean engry : ", np.mean(engry)) 
coef  = []
confirn = False
st = 0
en = 0
for i in range(1, len(engry)):
    coef_temp = int(engry[i] / engry[-1] )
    print(i, coef_temp)
    coef.append(coef_temp)
count = 0
for i in range(len(coef)):
    if coef[i] > 15:
        count += 1
    elif count > 1: 
        st = (i - count) * 0.07
        en = (i - 1) * 0.07
        print(st, en)
        st = int(st * f_sound)
        en = int(en * f_sound)
        shot_voice.append(sound[st:en])
        count = 0
print("Number of shot voice cuted : ", len(shot_voice))

Engry of 0.15:  [0.0, 9.4258474e-05, 0.00016822664, 0.00019899072, 0.00020500162, 0.0009458928, 0.0012716372, 0.00053559087, 0.00040796644, 0.0003995424, 0.000604836, 0.00048103815, 0.00033326918, 0.00059866655, 0.00124444, 0.001476106, 0.0014943916, 0.0009767382, 0.0005084163, 0.0011696683, 0.0012877949, 0.0007128729, 0.020980157, 0.08069508, 7.090887, 11.202526, 15.069609, 11.3773365, 10.310874, 4.3706193, 0.49096695, 0.0028221821, 0.0037126942, 3.9365952, 37.794983, 54.319736, 17.707539, 6.315042, 0.36618155, 0.0050117373, 0.029245516, 0.18499118, 3.725681, 4.877361, 3.0827408, 0.36101478, 0.003365292, 0.0006435408, 0.22680266, 4.0590825, 7.329992, 1.0010991, 0.50683177, 0.0063862517, 0.0010264728, 0.0009084638, 8.1658535, 25.267023, 25.6669, 5.7705207, 0.13481688, 0.002015217, 0.0010372831, 10.123291, 59.19854, 90.61711, 41.15404, 10.782955, 1.8481028, 0.014208645, 0.0021321194, 0.015629444, 1.449535, 5.1880827, 0.101645954, 0.059049897, 0.0015625301, 0.011098344, 0.06949293, 11.85

In [48]:
shot_voice = np.asanyarray(shot_voice)
X_test = []
y_test = np.array([0,1,2,3,4,5,6,7,8,9])
for i in shot_voice:
    feature = np.mean(lib.feature.mfcc(y = i, sr= f_sound,n_mfcc= 40).T, axis = 0)
    X_test.append(feature)
X_test = np.asanyarray(X_test)
y_pre = mlp.predict(X_test)
print(y_pre)
from sklearn.metrics import accuracy_score
print("Hieu qua mo hinh dat : ", 100* accuracy_score(y_test, y_pre.tolist() ),"%" )

[4 4 4 0 4 5 5 4 8 5]
Hieu qua mo hinh dat :  30.0 %
